## Imports

In [1]:
import itertools
from copy import deepcopy
from typing import Callable, List

import networkx as nx

from aoc_utilities import puzzle_input

<h2>--- Day 12: Passage Pathing ---</h2><p>With your <span title="Sublime.">submarine's subterranean subsystems subsisting suboptimally</span>, the only way you're getting out of this cave anytime soon is by finding a path yourself. Not just <em>a</em> path - the only way to know if you've found the <em>best</em> path is to find <em>all</em> of them.</p>
<p>Fortunately, the sensors are still mostly working, and so you build a rough map of the remaining caves (your puzzle input). For example:</p>
<pre><code>start-A
start-b
A-c
A-b
b-d
A-end
b-end
</code></pre>
<p>This is a list of how all of the caves are connected. You start in the cave named <code>start</code>, and your destination is the cave named <code>end</code>. An entry like <code>b-d</code> means that cave <code>b</code> is connected to cave <code>d</code> - that is, you can move between them.</p>
<p>So, the above cave system looks roughly like this:</p>
<pre><code>    start
    /   \
c--A-----b--d
    \   /
     end
</code></pre>
<p>Your goal is to find the number of distinct <em>paths</em> that start at <code>start</code>, end at <code>end</code>, and don't visit small caves more than once. There are two types of caves: <em>big</em> caves (written in uppercase, like <code>A</code>) and <em>small</em> caves (written in lowercase, like <code>b</code>). It would be a waste of time to visit any small cave more than once, but big caves are large enough that it might be worth visiting them multiple times. So, all paths you find should <em>visit small caves at most once</em>, and can <em>visit big caves any number of times</em>.</p>
<p>Given these rules, there are <code><em>10</em></code> paths through this example cave system:</p>
<pre><code>start,A,b,A,c,A,end
start,A,b,A,end
start,A,b,end
start,A,c,A,b,A,end
start,A,c,A,b,end
start,A,c,A,end
start,A,end
start,b,A,c,A,end
start,b,A,end
start,b,end
</code></pre>
<p>(Each line in the above list corresponds to a single path; the caves visited by that path are listed in the order they are visited and separated by commas.)</p>
<p>Note that in this cave system, cave <code>d</code> is never visited by any path: to do so, cave <code>b</code> would need to be visited twice (once on the way to cave <code>d</code> and a second time when returning from cave <code>d</code>), and since cave <code>b</code> is small, this is not allowed.</p>
<p>Here is a slightly larger example:</p>
<pre><code>dc-end
HN-start
start-kj
dc-start
dc-HN
LN-dc
HN-end
kj-sa
kj-HN
kj-dc
</code></pre>
<p>The <code>19</code> paths through it are as follows:</p>
<pre><code>start,HN,dc,HN,end
start,HN,dc,HN,kj,HN,end
start,HN,dc,end
start,HN,dc,kj,HN,end
start,HN,end
start,HN,kj,HN,dc,HN,end
start,HN,kj,HN,dc,end
start,HN,kj,HN,end
start,HN,kj,dc,HN,end
start,HN,kj,dc,end
start,dc,HN,end
start,dc,HN,kj,HN,end
start,dc,end
start,dc,kj,HN,end
start,kj,HN,dc,HN,end
start,kj,HN,dc,end
start,kj,HN,end
start,kj,dc,HN,end
start,kj,dc,end
</code></pre>
<p>Finally, this even larger example has <code>226</code> paths through it:</p>
<pre><code>fs-end
he-DX
fs-he
start-DX
pj-DX
end-zg
zg-sl
zg-pj
pj-he
RW-he
fs-DX
pj-RW
zg-RW
start-pj
he-WI
zg-he
pj-fs
start-RW
</code></pre>
<p><em>How many paths through this cave system are there that visit small caves at most once?</em></p>


## Solution
Part 1 is _almost_ asking us to count up the paths between a start and end node in the graph $T$ whose nodes are caves and edges tunnels in a given tunnel system.
I say "almost" because we are to include all walks that pass through "small cave" nodes once but may pass through big caves multiple times.
I'll call such a path **small cave-simple**.

In my solution, I decided to force this problem to be about simple paths and use NetworkX rather than reimplement a slight variation of the usual breadth-first search algorithm for simple paths.
To do this, I noted that this problem is equivalent to finding all simple paths in the multigraph $T_\text{small}$ that is obtained from $T$ by adding edges between each pair of nodes adjacent to a big cave node $n$ in $T$ and then removing $n$.

In [2]:
def all_small_cave_simple_edge_paths(
    G: nx.Graph, start='start', end='end'
):
    G_small = nx.MultiGraph(deepcopy(G))

    big_caves = [cave for cave in G_small.nodes if cave.isupper()]
    for big_cave in big_caves:
        for edge in itertools.combinations(G_small[big_cave], 2):
            G_small.add_edge(*edge, key=big_cave)
        G_small.remove_node(big_cave)

    yield from nx.simple_paths.all_simple_edge_paths(G_small, start, end)

### Testing

In [3]:
example_tunnel_logs = '''
    fs-end he-DX fs-he start-DX pj-DX end-zg zg-sl zg-pj pj-he
    RW-he fs-DX pj-RW zg-RW start-pj he-WI zg-he pj-fs start-RW
'''.split()

ET = nx.from_edgelist(tunnel.split('-') for tunnel in example_tunnel_logs)

assert len(list(all_small_cave_simple_edge_paths(ET))) == 226

### Answer

In [4]:
T = nx.from_edgelist(
    tunnel.split('-')
    for tunnel in puzzle_input.as_text(day=12).split('\n')
)

print(
    'Number of small cave-simple paths through the tunnel system:',
    len(list(all_small_cave_simple_edge_paths(T)))
)

Number of small cave-simple paths through the tunnel system: 4338


<h2 id="part2">--- Part Two ---</h2><p>After reviewing the available paths, you realize you might have time to visit a single small cave <em>twice</em>. Specifically, big caves can be visited any number of times, a single small cave can be visited at most twice, and the remaining small caves can be visited at most once. However, the caves named <code>start</code> and <code>end</code> can only be visited <em>exactly once each</em>: once you leave the <code>start</code> cave, you may not return to it, and once you reach the <code>end</code> cave, the path must end immediately.</p>
<p>Now, the <code>36</code> possible paths through the first example above are:</p>
<pre><code>start,A,b,A,b,A,c,A,end
start,A,b,A,b,A,end
start,A,b,A,b,end
start,A,b,A,c,A,b,A,end
start,A,b,A,c,A,b,end
start,A,b,A,c,A,c,A,end
start,A,b,A,c,A,end
start,A,b,A,end
start,A,b,d,b,A,c,A,end
start,A,b,d,b,A,end
start,A,b,d,b,end
start,A,b,end
start,A,c,A,b,A,b,A,end
start,A,c,A,b,A,b,end
start,A,c,A,b,A,c,A,end
start,A,c,A,b,A,end
start,A,c,A,b,d,b,A,end
start,A,c,A,b,d,b,end
start,A,c,A,b,end
start,A,c,A,c,A,b,A,end
start,A,c,A,c,A,b,end
start,A,c,A,c,A,end
start,A,c,A,end
start,A,end
start,b,A,b,A,c,A,end
start,b,A,b,A,end
start,b,A,b,end
start,b,A,c,A,b,A,end
start,b,A,c,A,b,end
start,b,A,c,A,c,A,end
start,b,A,c,A,end
start,b,A,end
start,b,d,b,A,c,A,end
start,b,d,b,A,end
start,b,d,b,end
start,b,end
</code></pre>
<p>The slightly larger example above now has <code>103</code> paths through it, and the even larger example now has <code>3509</code> paths through it.</p>
<p>Given these new rules, <em>how many paths through this cave system are there?</em></p>

## Solution

And now on Day 12 I've at last been burned by trying to be cute/clever with our solutions!
I can't think of a clever way to translate this problem into one supported by NetworkX out-of-the-box, so we'll roll our own breadth-first path generator here that'll work for Parts 1 and 2.
NetworkX makes the solution nice and readable.

In [5]:
def valid_walks(
    G: nx.Graph,
    valid_next_node_rule: Callable[[List[str], str], bool],
    source='start',
    target='end'
) -> List[List[str]]:
    partial_walks = [[source]]

    while partial_walks:
        partial_walk = partial_walks.pop()
        last_node_in_walk = partial_walk[-1]

        if target in G.neighbors(last_node_in_walk):
            yield partial_walk + [target]

        partial_walks.extend(
            partial_walk + [neighbor]
            for neighbor in G.neighbors(last_node_in_walk)
            if neighbor not in {source, target}
                and valid_next_node_rule(partial_walk, neighbor)
        )

Nice and readable, although it could almost certainly be optimized for time and space efficiency by converting to a memoized tail-recursive function that doesn't actually store the paths.
(Despite my role playing based on the prompt, we need the number of paths from `'start'` to `'end'` in order to solve the puzzle, not the paths themselves.)
Let's confirm this works to solve Part 1.

In [6]:
%%timeit
def unvisited_if_small_cave(walk: list[str], new_node) -> bool:
    return new_node.isupper() or new_node not in walk


assert len(list(valid_walks(ET, unvisited_if_small_cave))) == 226
assert len(list(valid_walks(T, unvisited_if_small_cave))) == 4338

42.4 ms ± 632 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Good and in a reasonable amount of time.
To solve Part 2, we just need to define a suitable `valid_next_node_rule` function.

In [7]:
def at_most_one_small_cave_visited_twice(walk, new_node):
    return (
        new_node.isupper()
        or new_node not in walk
        or (
            new_node in walk
            and max(walk.count(node) for node in walk if node.islower()) == 1
        )
    )

### Testing

In [8]:
%%timeit
assert len(list(valid_walks(ET, at_most_one_small_cave_visited_twice))) == 3509

142 ms ± 656 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Answer

In [9]:
print(
    'Number of specified walks through the tunnel system:',
    len(list(valid_walks(T, at_most_one_small_cave_visited_twice)))
)

Number of specified walks through the tunnel system: 114189


### Comment
Predictably, the performance tanks when generating the final solution, but it's good enough for solving the puzzle.
I'm sure there are other more optimized answers out there for anyone interested.

In [10]:
%%timeit
list(valid_walks(T, at_most_one_small_cave_visited_twice));

4.78 s ± 26 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
